<div>
<img src="https://www.nebrija.com/images/logos/logotipo-universidad-nebrija.jpg" width="200">
</div>

**MODELOS DE PROGRAMACION: MODELO ADIABATICO** -
Prof: Carmen Pellicer Lostao

# SAT PROBLEMS

## Intro

The D-Wave quantum computer is well suited to solving optimization and [satisfiability](https://en.wikipedia.org/wiki/Boolean_satisfiability_problem) (SAT) problems with binary variables. Binary variables can only have values 0 (NO, or FALSE) or 1 (YES, or TRUE), which are easily mapped to qubit final states of “spin up” (↑
) and “spin down” (↓). These are problems that answer questions such as, “Should a package be shipped on this truck?”


In [5]:
import dwave

## 2-SAT

### The 2-SAT problem

This section uses an example problem of a very simple SAT:

$(x_1 ∨ x_2)∧( \~x_1 ∨ \~x_2)$

or in another notation:
$(x_1 ∨ x_2)∧(\neg x_1 ∨ ¬x_2)$

This is a 2-SAT in **conjunctive normal** form (CNF), meaning it has disjunctions of two Boolean variables (X OR Y) joined by an AND, and is satisfied only if all its disjunctions are satisfied; that is, to make the SAT true, you must find and assign appropriate values of 0 or 1 to all its variables.

You can easily verify that the solution to this example problem is $x_1≠x_2$ or $x_1+x_2=1$

For example:

$x_1=x_2=0 → (0∨0)∧(1∨1)=0∧1=0$  (False)

$x_1=0 ≠ x_2=1 → (0∨1)∧(1∨0)=1∧1=1$ (True)

The objective function to find the solution to this 2-SAT problem is found after building a **Truth Table** where we asign the minimum Energy to solution states that satisfy our problem

A two-variable QUBO can have four possible values of its variables, representing four possible states of the system, as shown herein the following table

|  states | $q_1$ |  $q_2$ | Energy   |
|---|---|---|---|
|  1 | 0  | 0 | 1 |
|  2 | 0  | 1 | 0 |
|  3 | 1  | 0 | 0 |
|  4 | 1  | 1 | 1 |

For an objective function to represent the example SAT (i.e., to reformulate the original problem as an objective function that can be solved through minimization by a D-Wave solver), it needs to favor states 1 and 4 over states 2 and 3. You do this by penalizing states that do not satisfy the SAT; that is, by formulating the objective such that those states have higher energy.

First, notice that when $q_1$ and $q_2$ both equal 0—state 1—the value of the objective function is 0 for any value of the coefficients. To favor this state, you should formulate the objective function to have a global minimum energy (the ground state energy of the system) equal to 0. Doing so ensures that state 1 is a good solution.
Second, you penalize states 2 and 3 relative to state 1. One way to do this is to set both a1 and a2 to a positive value such as 0.11. Doing so sets the the value of the objective function for those two states to 0.10.1.

The QUBO function is the following:

$ E(q_i)= 1- q_1 - q_2 +2 q_1*q_2$

### Building the BQM

In `dimod` package we can find different [models](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/intro/intro_models.html). We can build a BQM from a QUBO or ISING funtion we need to pass the linear and quadratic terms in a dictionary format and the constant offset.

See the [BQM documentation](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/models.html#id10) to learn about the different ways to build a BQM object. Here we explore two ways `from_qubo()` method and `BinaryQuadraticModel()` constructor. In both ways we build the same BQM object and store it in a variable with the same name

In [6]:
from dimod import BinaryQuadraticModel

linear = {('q_1', 'q_1'): -1, ('q_2', 'q_2'): -1}
quadratic = {('q_1', 'q_2'): 2}

Q = {**linear, **quadratic}
bqm = BinaryQuadraticModel.from_qubo(Q, 1.0 )   #Create BQM object with method from_qubo() setting an offset to 1
print(bqm)

BinaryQuadraticModel({'q_1': -1.0, 'q_2': -1.0}, {('q_2', 'q_1'): 2.0}, 1.0, 'BINARY')


In [8]:
#The BQM parameters we see in the above BQM built from Q QUBO matrix 
linear1={'q_1': -1, 'q_2': -1}  # note linear information has a different format
quadratic = {('q_1', 'q_2'): 2}

#Create BQM object with BinaryQuadraticModel constructor and setting an offset to 1
bqm = BinaryQuadraticModel(linear1,quadratic, 1.0,'BINARY')     #BinaryQuadraticModel(linear, quadratic, offset, vartype) from linear and quadratic biases and an offset.
print(bqm)

BinaryQuadraticModel({'q_1': -1.0, 'q_2': -1.0}, {('q_2', 'q_1'): 2.0}, 1.0, 'BINARY')


### Solving the BQM in the QPU

To solve the 2-SAT problen in a QPU we use DWave quantum solver and automatic embedding. When we use a sampler from Ocean software’s dwave-system to solve on a D-Wave system, in addition to `DWaveSampler()`, we must use `EmbeddingComposite()`, which maps unstructured problems to the graph structure of the selected sampler, a process known as **minor-embedding**: our problem graph must be mapped to the QPU’s numerically indexed qubits.

We need to import the libraries and create a sampler

In [7]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())

See the documentation to learn about the different ways to sample a BQM objects. Here we explore two ways `sample_qubo()` and `sample` methods of sampler objects. In the first one we pass a QUBO matrix and in the second one a BQM object.

In [ ]:
#you can sample the QUBO matrix with the sampler
sampleset = sampler.sample_qubo(Q, num_reads=5000, label='SAT x1=x2 with matrix Q')
print(sampleset)

In [ ]:
#you can sample a BQM object with the sampler
sampleset = sampler.sample(bqm, num_reads=5000,label='SAT x1=x2 with bqm ans offset')
print(sampleset)

**Exercice:**

Justify the difference in the values ​​of the Energies in each case.

Observe that it is possible that some results from the sampled values ​​are not the optimal minimum.

For more efficient sampling, scaling the QUBO function is important in the solutions to be sampled.

In [ ]:
# Define QUBOs matrix with diferent scaling properties
#linear = {('q_1', 'q_1'): -1, ('q_2', 'q_2'): -1}
#quadratic = {('q_1', 'q_2'): 2}
Q1 = {('q1', 'q1'): -1.0, ('q2', 'q2'): -1.0, ('q1', 'q2'): 2}
Q2 = {('q1', 'q1'): -0.5, ('q2', 'q2'): -0.5, ('q1', 'q2'): 1}

#With autoscaling (the default),
# Autoscaling on for original QUBO
sampleset1 = sampler.sample_qubo(Q1, num_reads=5000)
print(sampleset1)
# Autoscaling on for scaled-up QUBO
sampleset2 = sampler.sample_qubo(Q2, num_reads=5000)
print(sampleset2)

#With autoscaling disabled,
# Autoscaling off for original QUBO
sampleset3 = sampler.sample_qubo(Q1, num_reads=5000, auto_scale=False)
print(sampleset3)
# Autoscaling off for scaled-up QUBO
sampleset4 = sampler.sample_qubo(Q2, num_reads=5000, auto_scale=False)
print(sampleset4)

To understand how our BQM is being executed in the QPU we can inspect the embedding in each case with the problem inspector libraries:

In [ ]:
import dwave.inspector

# Inspect the problems sampleset1,...,sampleset4
dwave.inspector.show(sampleset4)

### Interesting tools



[PyQUBO](https://pyqubo.readthedocs.io/en/latest/) allows you to create QUBOs or Ising models from flexible mathematical expressions easily.

You can convert a QUBO or Ising object generated qith pyQUBO to a DWave Binary Quadratic Model (BQM) object

In [1]:
import pyqubo as pq
q_1,q_2 = pq.Binary('q_1'), pq.Binary('q_2')
H = 1- q_1 -q_2 +2*q_1*q_2
qubo=H.compile().to_qubo()
print(qubo)

bqm=H.compile().to_bqm()
print(bqm)

ModuleNotFoundError: No module named 'pyqubo'

The utilities of the `dimod` package allow us to easily perform many operations such as model conversion, model generation or results management. Check out the `dimod` [utilities documentation](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/utilities.html) to see everything that can be done.

In [ ]:
import dimod

#Converting Between Models utilities
#qubo_to_ising
imodel=dimod.qubo_to_ising({('qb', 'qb'): -1, ('qg', 'qg'): -1, ('qb', 'qg'): 2})
print(imodel)

#ising_to_qubo
qmodel=dimod.ising_to_qubo({'qb': 0.0, 'qg': 0.0}, {('qb', 'qg'): 0.5}, -0.5)
print(qmodel)

In [ ]:
from dimod.generators import or_gate, and_gate
# generating BQM models

#generate a BQM  with variables x1, x2, x3, x4 that exactly 2 of the four variables are 1 satisfy the condition of min energy
bqm_4_2 = dimod.generators.combinations(['x1', 'x2', 'x3', 'x4'], 2)

#generate a BQM for 3 variables that satisfy the OR operation x1vx2=x3
bqm_or = or_gate('x1', 'x2', 'x3')

#generate a BQM for 3 variables that satisfy the AND operation x1^x2=x3
bqm_or = and_gate('x1', 'x2', 'x3')



Managing the [samples](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/intro/intro_samples.html)

In [ ]:
#Managing samples
sampleset = sampler.sample(bqm, num_reads=5000,label='SAT x1+x2=1 with bqm and offset')
print(sampleset)

We can build a pandas dataframe to store the sampler result

In [ ]:
df=sampleset.to_pandas_dataframe()
df.head()

Pandas allows us an easy and convenient way to manage ans query data. It is also very useful to store results in files so that we don't need to execute the sampler again and save QPU time

In [ ]:
!pip install openpyxl

In [ ]:
import openpyxl
import pandas as pd
df.to_excel('results.xlsx') #install  excel viewer complement to open excel files in VSCode
df.to_csv('results.csv')

The package `dimod` has also utilities to deal with **Non-Quadratic (Higher-Degree) Polynomials** reducing problems with interactions between more than pairs of variables to a QUBO.

This is done by introducing and minimizing over ancillary variables. A good procedure is to select the pair of variables that is most common amongst all terms with more than pairwise interactions, and replace it with an ancillary variable in all those terms. Repeat until no more terms of 3 or more variables remain. Note that subsequent iterations may involve products of ancillary variables if these ancillary pairs are the most common amongst the terms.

An interaction involving 4 or more variables can be reduced to pairwise by sequentially introducing new variables and reducing the degree of the interaction by 1 at each step.

Ocean software `dimod` pakage can automate such reduction for you

In [ ]:
cubic = {('x',): -1, ('y',): 1, ('z',): 1.5, ('x', 'y'): -1, ('x', 'y', 'z'): -2}
print(dimod.make_quadratic(cubic, 5, dimod.BINARY))

## 3-SAT

### Additional exercices


Make the QUBO for 3 variables that satisfy the following clause:

$\sum_i x_i = 1$

Execute in the QPU and check the embedings with the problem inspector tool

In [10]:
from dwave.system import DWaveSampler, EmbeddingComposite
import dwave.inspector

Q = {('q1', 'q2'): 2.0, ('q1', 'q3'): 2.0, ('q2', 'q3'): 2}

sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample_qubo(Q, num_reads=5000)

dwave.inspector.show(sampleset)

RuntimeError: threads can only be started once